In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt

In [2]:
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
DEVICE = torch.device("cpu")

test_acc_history = []
test_loss_history = []
train_acc_history = []
train_loss_history = []

True


In [3]:
train_data = datasets.CIFAR100('./data',
                     train=True,
                     download=True,
                     transform=transforms.Compose([
                         transforms.RandomCrop(32, padding=4),
                         transforms.RandomHorizontalFlip(),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))
                     ]))

test_data = datasets.CIFAR100('./data',
                     train=False,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))
                     ]))

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        #self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(16*5*5, 220)
        self.fc2 = nn.Linear(220, 184)
        self.fc3 = nn.Linear(184, 100)

    def forward(self, x):
        # x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.dropout(x, training=self.training)
        
        x = self.fc3(x)

        return F.log_softmax(x)

In [5]:
class my_CNN(nn.Module):
    def __init__(self):
        super(my_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 15, kernel_size=5)
        self.conv2 = nn.Conv2d(15, 40, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()

        self.fc1 = nn.Linear(5*5*40, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 100)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 5*5*40)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)

        return x

In [6]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)
model

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=220, bias=True)
  (fc2): Linear(in_features=220, out_features=184, bias=True)
  (fc3): Linear(in_features=184, out_features=100, bias=True)
)

In [7]:
def train(model, train_loader, optimizer, epoch):
    train_loss, train_accuracy, running_train_loss, running_train_correct=0,0,0,0

    model. train()

    for batch_idx, (data, target) in enumerate(train_loader):
        running_loss = 0
        data, taget = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.cross_entropy(output, target)
        loss.backward()

        optimizer.step()

        _, pred = torch.max(output.data, 1)
        running_train_correct += (pred == target).sum().item()
        running_train_loss += loss.item()
    train_loss = running_train_loss / len(train_loader)
    train_accuracy = (running_train_correct / len(train_loader.dataset) * 100)

    return train_loss, train_accuracy

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss=0
    correct=0

    with torch.no_grad():
        for data, target in test_loader:
            data, taget = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss+=F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

In [9]:
def CNN_test(epoch, batch):
    Epoch = epoch
    Batch_Size = batch

    train_loader=torch.utils.data.DataLoader(
        train_data, batch_size=Batch_Size, shuffle=True
    )
    test_loader=torch.utils.data.DataLoader(
        test_data, batch_size=Batch_Size, shuffle=True
    )
    

    for epoch in tqdm(range(1, Epoch+1)):
        train_loss, train_accuracy = train(model, train_loader, optimizer, epoch)
        test_loss, test_accuracy = evaluate(model, test_loader)

        print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
            epoch, test_loss, test_accuracy
        ))
        test_acc_history.append(test_accuracy)
        test_loss_history.append(test_loss)
        train_acc_history.append(train_accuracy)
        train_loss_history.append(train_loss)

In [ ]:
CNN_test(100, 64)

  1%|          | 1/100 [00:30<50:14, 30.45s/it]

[1] Test Loss: 4.6030, Accuracy: 0.95%


  2%|▏         | 2/100 [01:01<50:46, 31.09s/it]

[2] Test Loss: 4.5771, Accuracy: 1.40%


  3%|▎         | 3/100 [01:32<49:31, 30.64s/it]

...

[10] Test Loss: 3.7642, Accuracy: 12.19%


 11%|█         | 11/100 [05:32<44:43, 30.15s/it]

[11] Test Loss: 3.6950, Accuracy: 14.09%


 12%|█▏        | 12/100 [06:02<44:16, 30.19s/it]

[12] Test Loss: 3.6227, Accuracy: 15.15%


In [ ]:
test_acc_ser = pd.Series(test_acc_history)
test_loss_ser = pd.Series(test_loss_history)
train_acc_ser = pd.Series(train_acc_history)
train_loss_ser = pd.Series(train_loss_history)

plt.plot(test_acc_ser)
plt.plot(train_acc_ser)
plt.show()

In [ ]:
plt.plot(test_loss_ser)
plt.plot(train_loss_ser)
plt.show()

In [ ]:
torch.save(model.state_dict(), './cnn_model.pt')
print('state_dict format of the model: {}'.format(model.state_dict()))